
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement du changement climat climatique ou l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, produit par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif du pojet à l'IFÉ ENS de Lyon](https://contrib-tremplin.ens-lyon.fr/) et une copie se trouve dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. 

Les ressources  du projet **peuvent être utilisées dans tout autre environnement compatible**, notamment tous les cahiers d'exercices peuvent être exécutés sur toute machine disposant d'un python3  et des bibliothèques jupyter, jupyterlab, numpy, netcdf4. 

Les données _pré-traitées_ utilisées ci-dessous sont **accessibles  en ligne** sur le [serveur de données géolocalisées](http://geoloc-tremplin.ens-lyon.fr/climato-data/) `opendap` du projet tremplin.

![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche des enseignements autour du changement climatique : mitigation et adaptation au changement

# _Phase 1 : Conversion d'un jeu de données extrait de DRIAS pour des traitements locaux et la production de graphiques ou cartes_

Ce cahier d'exercices `ipython` propose une méthode de conversion d'un jeu de données issues des simulations climatiques de Météofrance. Le travail réalisé ici concerne l'explitation de trois séries de modélisations du CNRM  (RCP 2.6 - 4.5 - 8.5).  Les fichiers commandés et téléchargés sur le site  [DRIAS](http://www.drias-climat.fr/), ne peuvent être (encore) directement servis par le serveur du projet (limitation du logiciel OpenDap). Ce cahier permet de convertir ce qui a été reçu de  [DRIAS](http://www.drias-climat.fr/) et de le convertir pour permettre leur stockage en ligne puis l'utilisation directe depuis le cloud à l'aide des autres cahiers de programmes proposés par le projet sur ce site. 

Les programmes ci-dessous **ne peuvent pas être complètement génériques** et une interface graphique permettant de fournir interactivement les paramètres Ad-Hoc n'a pas encore été développée. Les extractions de [DRIAS](http://www.drias-climat.fr/) sont parfaitement cohérentes  et il suffit pour réutiliser ce cahier d'exercices :

    1. De repérer les **noms des répertoires** où se trouvent les fichiers originaux de Drias, et le répertoire où seront sauvegardés les fichiers transformés et de les remplacer.
    2.  De repérer les **noms des fichiers à traiter** et de les remplacer
    3. De repérer le **nom de la variable principale** et de le remplacer si besoin (À cette fin un bloc de programmes permettant d'imprimer  les paramètres de chaque fichier est proposé il suffit de décommenter les lignes  réalisant une impression).

Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Les données concernent un carré 10 x 10 noeuds de la grille ALADIN centré sur la ville de Lyon. Le format d'entrée et de sortie est celui utilisé par les labos de climatologie, d'océanographie : `netCDF`.  Cet outil requiert l'installation des bibliothèques `netCDF4` et `numpy`.

## Préparation de l'environnement et ouverture du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [1]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, os
import multiprocessing as mp

## Traitement des données par variables

Importation des données depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise, construction d'un jeu de données en convertissant la totalité des variables codées en 64 bits vers un codage sur 32 bits pour que le serveur opendap les gère convenablement.

Le bloc ci dessous contient la fonction qui modifie le type dela variable et récupère les autres paramètres. Ce code est conçu pour exploiter les extractions standard du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp). 

In [2]:
def convertFile(localPathIn, localPathOut, names) :

    driasFileName = names[0]
    driasVarName = names[1]
    namein =  localPathIn + driasFileName
    nameout = localPathOut + driasFileName
    #print (namein, '\n',  nameout,'\n\n')
    # open input file
    origDrias = nc.Dataset(namein)
    # open output file
    try : os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles   
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4')
    #print ('done', i)
    # create dimensions
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        convertedSet.createDimension(thisDim.name, thisDim.size)
    # create variables
    newOutVar = convertedSet.createVariable(driasVarName, 'f4', ('time','j','i'),fill_value=1.e+20)
    for thisAttr in origDrias.variables[driasVarName].ncattrs() :
        newOutVar.setncattr(thisAttr,origDrias.variables[driasVarName].getncattr(thisAttr))
    newOutVar[:,:,:] = origDrias.variables[driasVarName][:,:,:] 
    # print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name =  origDrias.variables['i'].long_name
    i[:] = origDrias.variables['i'][:]
    # print('\t i :',i)
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = origDrias.variables['j'].long_name
    j[:] = origDrias.variables['j'][:]
    # print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units =  origDrias.variables['lat'].units
    lat.long_name = origDrias.variables['lat'].long_name
    lat.standard_name = origDrias.variables['lat'].standard_name
    lat._CoordinateAxisType = origDrias.variables['lat']._CoordinateAxisType
    lat[:,:]  = origDrias.variables['lat'][:,:] 
    # print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = origDrias.variables['lon'].units
    lon.long_name = origDrias.variables['lon'].long_name
    lon.standard_name = origDrias.variables['lon'].standard_name
    lon._CoordinateAxisType = origDrias.variables['lon']._CoordinateAxisType
    lon[:,:] = origDrias.variables['lon'][:,:]
    # print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = origDrias.variables['time'].units
    time.long_name = origDrias.variables['time'].long_name
    time.standard_name = origDrias.variables['time'].standard_name
    time.calendar = origDrias.variables['time'].calendar
    time.axis = origDrias.variables['time'].axis
    time[:] = origDrias.variables['time'][:]
    # print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = origDrias.variables['x'].units
    x.long_name = origDrias.variables['x'].long_name
    x.standard_name = origDrias.variables['x'].standard_name
    x[:] = origDrias.variables['x'][:]
    # print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = origDrias.variables['y'].units
    y.long_name = origDrias.variables['y'].long_name
    y.standard_name = origDrias.variables['y'].standard_name
    y[:] = origDrias.variables['y'][:]
    # print('\t y :',y)
    
    for thisAttr in  origDrias.ncattrs() :
        convertedSet.setncattr(thisAttr,origDrias.getncattr(thisAttr))

    convertedSet.close()
    print(driasFileName, '\t\t',datetime.now())

    return (i,driasFileName)

## Programme principal 

Dans l'attente de mise en place d'interactivité avec `ipywidgets` ou `flexx` le programme principal reçoit le variables permettant d'effectuer le calcul :
    * Le chemin du répertoire contenant les fichiers issus de [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp)
    * Le chemin du répertoire d'accueil des fichiers convertis (ils portent le même nom que les fichiers d'origine ils doivent donc être stockés dans un autre répertoire
    * Un tableau contenant le nom du fichier et le nom de la variable, par défaut on conserve le nom original mais ce tableau permet éventuellement d'utiliser des noms Ad-Hoc.
    
Sous le bloc de code sont imprimés les noms des fichiers traités et leur horodatation.

In [5]:
i = 0

#pathIn = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/'
#pathOut = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/'
pathIn = 'E:\\Tremplin\\ClimatLyon\\DataDrias\\Toulouse-1\\'
pathOut = r'E:\\Tremplin\\ClimatLyon\\ConvertedDrias\\Toulouse-1\\'

Names = [['huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc','huss'],
            ['huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc','huss'],
            ['huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc','huss'],
            ['huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc','huss'],
            ['prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc','prsnls'],
            ['prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc','prsnls'],
            ['prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc','prsnls'],
            ['prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc','prsnls'],
            ['rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc','rstr'],
            ['rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc','rstr'],
            ['rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc','rstr'],
            ['rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc','rstr'],
            ['tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc','tasmax'],
            ['tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc','tasmax'],
            ['tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc','tasmax'],
            ['tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc','tasmax'],
            ['tasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc','tasmin'],
            ['tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc','tasmin'],
            ['tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc','tasmin'],
            ['tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc','tasmin']] 

for  i, thisName in  enumerate(Names) :   
    convertFile(pathIn, pathOut, thisName)
    i += 1


huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-15 00:49:43.208490
huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-15 00:49:50.833490
huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-15 00:49:58.239740
huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 		 2019-07-15 00:50:06.208490
prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-15 00:50:10.802240
prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-15 00:50:18.614740
prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-15 00:50:26.364740
prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 		 2019-07-15 00:50:34.052240
rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 		 2019-07-15 00:50:38.864740
rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 		 2019-07-15 00:50:46.692865
rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 		 2019-07-15 00:50:54.427240
rst

## Facultatif
Ce bloc permet de vérifier les noms fournis en sortie par le traitement.